In [1]:
import pandas as pd
import altair as alt

DATA_FOLDER = "../data"

 # Data exploration

 Quick notebook for exploring the `movies.csv` dataset

 Importing the dataset and listing columns:

In [2]:
movies = pd.read_csv(f"{DATA_FOLDER}/movies.csv").convert_dtypes()
movies.columns

Index(['id', 'title', 'tagline', 'description', 'genres', 'keywords', 'date',
       'collection', 'runtime', 'revenue', 'budget', 'director', 'cast',
       'production_companies', 'production_countries', 'popularity',
       'average_vote', 'num_votes', 'language', 'imdb_id', 'poster_url'],
      dtype='object')

 Plotting number of missing values per column:

In [3]:
missing = movies.isna().sum(axis=0).reset_index(name="count")

alt.Chart(missing, title="Missing values in columns").mark_bar().encode(
    x=alt.X(
        "index:N",
        title="Columns",
        sort=alt.SortField(field="count", order="descending"),
    ),
    y=alt.Y("count:Q", title="Count"),
)

alt.Chart(...)

## Duplicated values

There are quite a few duplicated values, thankfully we can find them through the *id* attribute

In [4]:
duplicated_table = (movies["id"].value_counts() > 1).reset_index()
duplicated_idx = duplicated_table[duplicated_table["id"]]["index"]
movies[movies["id"] == duplicated_idx.iloc[-1]]

,id,title,tagline,description,genres,keywords,date,collection,runtime,revenue,...,director,cast,production_companies,production_countries,popularity,average_vote,num_votes,language,imdb_id,poster_url
36378,289190,Felt,<NA>,A women creates an alter ego in hopes of overc...,"drama, romance, horror",<NA>,2014-09-18,<NA>,80,0,...,Jason Banker,"Amy Everson, Kentucker Audley, Alanna Reynolds...",<NA>,<NA>,0.643327,5.0,9,en,tt3854104,/z9WRZdKXmGNTBXPe3JbNOcoGrvh.jpg
36379,289190,Felt,<NA>,A women creates an alter ego in hopes of overc...,"drama, romance, horror",<NA>,2014-09-18,<NA>,80,0,...,Jason Banker,"Amy Everson, Kentucker Audley, Alanna Reynolds...",<NA>,<NA>,0.643327,5.0,9,en,tt3854104,/z9WRZdKXmGNTBXPe3JbNOcoGrvh.jpg


In the original dataset, all duplicated rows with the same *id* have the same value for each other column, but we can check this.

In [5]:
def check_identical_df(df: pd.DataFrame) -> bool:
    """Checks whether a given dataframe contains identical rows"""

    test_row = df.iloc[0]
    return ((test_row.eq(test_row)).all()).all()

check_all_duplicates = (
    movies[movies["id"].isin(duplicated_idx)]
    .groupby("id")
    .apply(check_identical_df)
).all()
check_all_duplicates

True

We can then create a `deduplicated_movies` dataset to remove all duplicates

In [6]:
if check_all_duplicates:
    deduplicated_movies = movies.groupby("id").first().reset_index()
    deduplicated_movies.to_csv(f"{DATA_FOLDER}/deduplicated_movies.csv", index=False, header=True)

 ## Genres

 The genres of the movies should be our target variable for the inital text classification. We're going to have a closer look:

In [7]:
movies_with_genres = deduplicated_movies[deduplicated_movies["genres"].notna()]
movies_with_genres["genres"].value_counts()

drama                                  4996
comedy                                 3620
documentary                            2721
drama, romance                         1300
comedy, drama                          1133
                                       ... 
animation, action, western                1
adventure, music, mystery, thriller       1
adventure, family, drama, comedy          1
drama, western, romance, action           1
animation, family, mystery                1
Name: genres, Length: 4065, dtype: Int64

 There are some genres that contain multiple genres for one film. We're going to split them out so that there are different rows with the same *id* and every other column but different genre, one of the list in the initial dataset.

In [8]:
def list_column_to_long_format(
    dataframe: pd.DataFrame, column: str, delimiter: str = ","
) -> pd.DataFrame:
    """Returns a dataframe where values in column are not delimiter-separated lists anymore but one per row, with all other information duplicated on multiple rows"""
    assert column in dataframe.columns, "Column must be in dataframe"
    
    # Expand Series of column to DataFrame and add index as id for merging afterwards
    splitted_columns = dataframe[column].str.split(delimiter, expand=True).reset_index()

    # Convert from wide format to long format and drop NAs
    long_format = (
        pd.melt(
            splitted_columns,
            id_vars="index",
            value_vars=splitted_columns.drop(columns="index").columns,
            value_name=column
        )
        .drop(columns="variable")
        .dropna()
    )

    # Strip all values
    long_format[column] = long_format[column].str.strip()

    # Merge dataframe
    merged_df = pd.merge(
        long_format, 
        dataframe.drop(columns=column).reset_index(), 
        on="index",
        how="inner"
    )
    return merged_df.drop(columns="index")

In [9]:
multiple_genres = movies_with_genres[movies_with_genres["genres"].str.contains(",")]
print(
    f"Percentage of multiple genre movies: {100 * len(multiple_genres) / len(movies_with_genres):.2f}%"
)

Percentage of multiple genre movies: 66.15%


In [10]:
movies_splitted_genres = list_column_to_long_format(deduplicated_movies, "genres")

 Now we can plot the number of different genres:

In [11]:
alt.Chart(
    movies_splitted_genres["genres"].value_counts().reset_index(name="count"),
    title="Genre Count",
).mark_bar().encode(
    x=alt.X("index:N", title="Genre", sort=alt.SortField("count", "descending")),
    y=alt.Y("count:Q", title="Number of films"),
)


alt.Chart(...)

## Keywords

We must also convert all lists of keywords to tuples:

In [12]:
movies = list_column_to_long_format(movies_splitted_genres, "keywords")

Number of different keywords:

In [13]:
movies["keywords"].unique().shape[0]

19933

We can order by frequency and have a look at the distribution of the top-100:

In [14]:
alt.Chart(movies["keywords"].value_counts()[:100].reset_index(), title="Keyword frequency").mark_bar().encode(
    x=alt.Y("index:N", sort=alt.SortField(field="keywords", order="descending")),
    y="keywords:Q"
)

alt.Chart(...)